In [63]:
import os
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [70]:
partial = pd.read_csv("dataset_clustered.csv", index_col = 0)
ctx = pd.read_csv("sent_ctx.csv", index_col = 0).drop('sentences', axis=1)

In [71]:
partial = partial.join(ctx, how='inner')

In [54]:
#partial.sort_index().to_csv("dataset_clustered.csv")

In [72]:
annotated = partial[partial['annotated'] == True]
annotated.head()

,sentences,greeting,misc:books,misc:hats,misc:balls,need:books,need:hats,need:balls,offer:books,offer:hats,offer:balls,inquiry,agreement,thanks,numeric,same,annotated,cluster,context
index,,,,,,,,,,,,,,,,,,,
0,i would like 4 hats and you can have the rest .,False,NaN,NaN,NaN,NaN,4,NaN,rest,rest,rest,NaN,NaN,False,NaN,False,True,26,1 4 1
1,deal,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Full,False,NaN,False,True,0,1 4 1
2,you can have all the hats if i get the book an...,False,NaN,NaN,NaN,1,NaN,all,NaN,all,NaN,NaN,NaN,False,NaN,False,True,14,1 3 2
3,hi i would like the book and ball and you can ...,True,NaN,NaN,NaN,1,NaN,1,NaN,all,NaN,NaN,NaN,False,NaN,False,True,23,1 3 1
4,ill take the book,False,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,True,-1,1 3 1


In [73]:
def get_item_str(row, prefix):
    items_str = ""
    n_books = str(row[f"{prefix}:books"]).strip()
    n_hats = str(row[f"{prefix}:hats"]).strip()
    n_balls = str(row[f"{prefix}:balls"]).strip()
    if n_books == 'rest' and n_hats == 'rest' and n_balls == 'rest':
        items_str = 'rest'
    else:
        if n_books and n_books != 'nan':
            items_str += f"_books {n_books} "
        if n_hats and n_hats != 'nan':
            items_str += f"_hats {n_hats} "
        if n_balls and n_balls != 'nan':
            items_str += f"_balls {n_balls}"
    return items_str

def make_template(annotated_row):
    template_str = ""
    if annotated_row['greeting']:
        template_str += "_greeting "
    
    if annotated_row['agreement'] == 'Full':
        template_str += "_deal "
    elif annotated_row['agreement'] == 'Yes':
        template_str += "_yes "
    elif annotated_row['agreement'] == 'No':
        template_str += "_no "        
    elif annotated_row['agreement'] == 'No Deal':
        template_str += "_nodeal "        
    elif annotated_row['agreement'] == 'Counter':
        template_str += "_counteroffer "   
        
    items_str = get_item_str(annotated_row, "misc")
    if len(items_str) > 0 :
        template_str += "_misc( " + items_str + ") "
    items_str = get_item_str(annotated_row, "need")
    if len(items_str) > 0 :
        template_str += "_need( " + items_str + ") "
    items_str = get_item_str(annotated_row, "offer")
    if len(items_str) > 0 :
        template_str += "_offer( " + items_str + ") "
    
    
    if annotated_row['thanks']:
        template_str += "_thanks "
    return template_str


In [74]:
annotated['template'] = annotated.apply(make_template, axis=1)

/home/ravy/anaconda3/envs/py379/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [75]:
annotated


,sentences,greeting,misc:books,misc:hats,misc:balls,need:books,need:hats,need:balls,offer:books,offer:hats,offer:balls,inquiry,agreement,thanks,numeric,same,annotated,cluster,context,template
index,,,,,,,,,,,,,,,,,,,,
0,i would like 4 hats and you can have the rest .,False,NaN,NaN,NaN,NaN,4,NaN,rest,rest,rest,NaN,NaN,False,NaN,False,True,26,1 4 1,_need( _hats 4 ) _offer( rest)
1,deal,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Full,False,NaN,False,True,0,1 4 1,_deal
2,you can have all the hats if i get the book an...,False,NaN,NaN,NaN,1,NaN,all,NaN,all,NaN,NaN,NaN,False,NaN,False,True,14,1 3 2,_need( _books 1 _balls all) _offer( _hats all )
3,hi i would like the book and ball and you can ...,True,NaN,NaN,NaN,1,NaN,1,NaN,all,NaN,NaN,NaN,False,NaN,False,True,23,1 3 1,_greeting _need( _books 1 _balls 1) _offer( _h...
4,ill take the book,False,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,True,-1,1 3 1,_need( _books 1 )
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30504,deal . you can have the book and i take the 3 ...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Full,False,NaN,False,True,7,1 3 1,_deal
30516,yes,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Full,False,NaN,False,True,9,1 3 3,_deal
30517,deal . thank you .,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Full,True,NaN,False,True,16,1 3 3,_deal _thanks


In [76]:
template_sample = annotated[annotated['template'] != '']
template_sample.shape

(7016, 20)

# Only Unique Sentences

In [77]:
#template_sample = template_sample.drop_duplicates(subset=['sentences'])

In [78]:
template_sample[['sentences', 'template', 'context']].sort_index().to_csv("data/template_sample.csv")

In [79]:
template_sample[['sentences', 'template', 'context']].sort_index().to_json("data/template_sample.json", orient='records')

In [80]:
train, test = train_test_split(template_sample, test_size=.2, random_state=678)

In [83]:
train[['sentences', 'template', 'context']].sort_index().to_csv("data/template_sample_train.csv")
test[['sentences', 'template', 'context']].sort_index().to_csv("data/template_sample_test.csv")

In [82]:
train[['sentences', 'template', 'context']].sort_index().to_json("data/template_sample_train.json", orient='records')
test[['sentences', 'template', 'context']].sort_index().to_json("data/template_sample_test.json", orient='records')